By C4

In [1]:
%pylab inline
import pandas as pd
import workalendar.europe.belgium as belgium

Populating the interactive namespace from numpy and matplotlib


In [2]:
raw_data = pd.read_csv("data/dataraw.csv", parse_dates=["time"])

In [3]:
cleaned_data = raw_data[["time", "is_open"]].copy()
cleaned_data.loc[cleaned_data.time.dt.hour >= 23, "is_open"] = False
cleaned_data.loc[cleaned_data.time.dt.hour < 7, "is_open"] = False
cleaned_data["date"] = cleaned_data.time.dt.date

cleaned_data.sample(10)

,time,is_open,date
28586,2016-03-12 16:00:00,True,2016-03-12
4119,2013-05-28 05:00:00,False,2013-05-28
35988,2017-01-15 02:00:00,False,2017-01-15
32784,2016-09-03 14:00:00,False,2016-09-03
25379,2015-10-31 01:00:00,False,2015-10-31
22130,2015-06-17 16:00:00,False,2015-06-17
27345,2016-01-20 23:00:00,False,2016-01-20
12076,2014-04-24 18:00:00,False,2014-04-24
22998,2015-07-23 20:00:00,False,2015-07-23
31084,2016-06-24 18:00:00,False,2016-06-24


In [4]:
data_dates = pd.Series(cleaned_data["date"].unique(), name="day")
data_dates.sample(10)

479     2014-03-31
476     2014-03-28
130     2013-04-16
894     2015-05-20
1142    2016-01-23
579     2014-07-09
559     2014-06-19
35      2013-01-11
1483    2016-12-29
480     2014-04-01
Name: day, dtype: object

In [5]:
starting_days = [
    datetime.date(2009, 9, 14),
    datetime.date(2010, 9, 20),
    datetime.date(2011, 9, 19),
    datetime.date(2012, 9, 17),
    datetime.date(2013, 9, 16),
    datetime.date(2014, 9, 15),
    datetime.date(2015, 9, 14),
    datetime.date(2016, 9, 19),
    datetime.date(2017, 9, 18),
]

assert all([d.weekday() == 0 for d in starting_days])

In [6]:
def get_week_no(date):
    possible = [d for d in starting_days if d <= date]
    aca_start = possible[-1]
    dt = date - aca_start
    return math.floor(timedelta64(dt) / timedelta64(1,'W'))

week_df = data_dates.to_frame()
week_df["week_no"] = week_df["day"].apply(get_week_no)
week_df["day"] = pd.to_datetime(week_df["day"])

#week_df[week_df.day.dt.date > datetime64("2013-09-10")].head(15)

In [7]:
featurized = cleaned_data.copy()

# day of the week
featurized["weekday"] = featurized["time"].dt.weekday

# academic week number
featurized["aca_week_no"] = featurized["date"].apply(get_week_no)

# civil week number
featurized["week_no"] = featurized["time"].dt.week

# date
featurized["year"] = featurized["time"].dt.year
featurized["month"] = featurized["time"].dt.month
featurized["day"] = featurized["time"].dt.day
featurized["hour"] = featurized["time"].dt.hour

# holiday
calendar = belgium.Belgium()
featurized["working"] = featurized["date"].apply(calendar.is_working_day)
### St V
featurized.loc[(featurized["time"].dt.month == 11) & (featurized["time"].dt.day == 20), "working"] = False

# tampon
featurized["tampon"] = False
featurized.loc[featurized.aca_week_no.isin([6, 13, 26, 34]), "tampon"] = True

# exams
featurized["exams"] = False
featurized.loc[featurized.aca_week_no.isin([14, 15, 16, 17, 18]), "exams"] = True
featurized.loc[featurized.aca_week_no.isin([36, 37, 38, 39, 40]), "exams"] = True

# vacances
featurized["vacances"] = False
featurized.loc[featurized.aca_week_no.isin([19] + list(range(41, 52))), "vacances"] = True

# weather

featurized.sort_values("time")
featurized.sample(5)

,time,is_open,date,weekday,aca_week_no,week_no,year,month,day,hour,working,tampon,exams,vacances
4318,2013-06-05 12:00:00,False,2013-06-05,2,37,23,2013,6,5,12,True,False,True,False
17937,2014-12-24 23:00:00,False,2014-12-24,2,14,52,2014,12,24,23,True,False,True,False
29435,2016-04-17 01:00:00,False,2016-04-17,6,30,15,2016,4,17,1,False,False,False,False
8608,2013-12-01 06:00:00,False,2013-12-01,6,10,48,2013,12,1,6,False,False,False,False
15086,2014-08-28 04:00:00,False,2014-08-28,3,49,35,2014,8,28,4,True,False,False,True


In [8]:
featurized.to_csv("data/data_featurized.csv")